In [2]:
from random import randint

class Dot: # описание точек
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
class BoardException(Exception):# описание классов
    pass

class BoardOutException(BoardException):
    def __str__(self):
        return "Точка находится вне доски"

class BoardUsedException(BoardException):
    def __str__(self):
        return "Вы уже сюда стреляли"
    
class BoardWrongShipException(BoardException):
    pass

class Ship: #описание класса корабля
    def __init__(self, bow, l, o): # параметр "о" задает ориентацию корабля
        self.bow = bow
        self.l = l
        self.o = o
        self.lives = l
        
        
    @property
    def dots(self):
        ship_dots = []
        for i in range(self.l):
            cur_x = self.bow.x 
            cur_y = self.bow.y
            
            if self.o == 0:
                cur_x += i
            
            elif self.o == 1:
                cur_y += i
            
            ship_dots.append(Dot(cur_x, cur_y))
        
        return ship_dots

    def shooten(self, shot): # координатор стрельбы
        return shot in self.dots

class Board:  # класс поля
    def __init__(self, hid = False, size = 6): #описание размера поля и чьё это поле true - человек, false - бот)
        self.size = size
        self.hid = hid
        
        self.count = 0 # количество пораженных кораблей
        
        self.field = [ ["O"]*size for _ in range(size) ]
        
        self.busy = []
        self.ships = []

    def __str__(self):  #игровая доска
        res = ""
        res += "  | 1 | 2 | 3 | 4 | 5 | 6 |"
        for i, row in enumerate(self.field):
            res += f"\n{i+1} | " + " | ".join(row) + " |"
        
        if self.hid:
            res = res.replace("$", "O")
        return res  #нужно ли скрывать корабли на доске (true, false)
    
    def out(self, d):   # проверка на принадлежность точки полю
        return not((0<= d.x < self.size) and (0<= d.y < self.size))
    
    def contour(self, ship, verb = False): # объявление всех точек, возле которых мы находимся
        near = [
            (-1, -1), (-1, 0) , (-1, 1),
            (0, -1), (0, 0) , (0 , 1),
            (1, -1), (1, 0) , (1, 1)
        ]
        for d in ship.dots:
            for dx, dy in near:
                cur = Dot(d.x + dx, d.y + dy)
                if not(self.out(cur)) and cur not in self.busy:
                    if verb:
                        self.field[cur.x][cur.y] = "."
                    self.busy.append(cur)
                    
    def add_ship(self, ship): #размещение корабля
        
        for d in ship.dots:
            if self.out(d) or d in self.busy:
                raise BoardWrongShipException()   # точки кораблей не выходят за границы и не заняты
        for d in ship.dots:
            self.field[d.x][d.y] = "$"
            self.busy.append(d)
        
        self.ships.append(ship)
        self.contour(ship)
    def shot(self, d):
        if self.out(d):
            raise BoardOutException()
        
        if d in self.busy:
            raise BoardUsedException()
        
        self.busy.append(d)
        
        for ship in self.ships:  # выстрел
            if d in ship.dots:
                ship.lives -= 1
                self.field[d.x][d.y] = "X"
                if ship.lives == 0:
                    self.count += 1
                    self.contour(ship, verb = True)  
                    print("Корабль уничтожен!")  # исключения
                    return False
                else:
                    print("Корабль ранен!")
                    return True
        
        self.field[d.x][d.y] = "."
        print("Мимо!")
        return False
    
    def begin(self):
        self.busy = []  # обнуление списка
        
class Player:  # класс игрока
    def __init__(self, board, enemy):  # доски игрока и компьютера
        self.board = board
        self.enemy = enemy
    
    def ask(self):
        raise NotImplementedError()
    
    def move(self):
        while True:
            try:
                target = self.ask()
                repeat = self.enemy.shot(target)
                return repeat
            except BoardException as e:
                print(e)
                
class AI(Player):  # классы игроков
    def ask(self):
        d = Dot(randint(0,5), randint(0, 5))  # генерация  точек
        print(f"Ход компьютера: {d.x+1} {d.y+1}")
        return d

class User(Player):
    def ask(self):
        while True:
            cords = input("Ваш ход: ").split()
            
            if len(cords) != 2:
                print(" Введите 2 координаты! ")
                continue
            
            x, y = cords
            
            if not(x.isdigit()) or not(y.isdigit()):
                print(" Введите числа! ")
                continue
            
            x, y = int(x), int(y)
            
            return Dot(x-1, y-1)
        
class Game:
    def __init__(self, size = 6):   # задание размера доски и генерация досок
        self.size = size
        pl = self.random_board()
        co = self.random_board()
        co.hid = True
        
        self.ai = AI(co, pl)
        self.us = User(pl, co)
    
    def random_board(self):
        board = None
        while board is None:
            board = self.random_place()
        return board
    
    def random_place(self):
        lens = [3, 2, 2, 1, 1, 1, 1]
        board = Board(size = self.size)
        attempts = 0
        for l in lens:
            while True:
                attempts += 1
                if attempts > 2000:
                    return None
                ship = Ship(Dot(randint(0, self.size), randint(0, self.size)), l, randint(0,1))
                try:
                    board.add_ship(ship)
                    break
                except BoardWrongShipException:
                    pass
        board.begin()
        return board

    def greet(self):   # приветствие
        print("-------------------")
        print("  Приветсвуем вас  ")
        print("      в игре       ")
        print("    морской бой    ")
        print("-------------------")
        print(" формат ввода: x y ")
        print(" x - номер строки  ")
        print(" y - номер столбца ")
        
    def loop(self):   # игровой цикл
        num = 0
        while True:
            print("-"*20)
            print("Доска пользователя:")
            print(self.us.board)
            print("-"*20)
            print("Доска компьютера:")
            print(self.ai.board)
            if num % 2 == 0:
                print("-"*20)
                print("Ходит пользователь!")
                repeat = self.us.move()
            else:
                print("-"*20)
                print("Ходит компьютер!")
                repeat = self.ai.move()
            if repeat:
                num -= 1
            
            if self.ai.board.count == 7:
                print("-"*20)
                print("Пользователь выиграл!")
                break
            
            if self.us.board.count == 7:
                print("-"*20)
                print("Компьютер выиграл!")
                break
            num += 1
    def start(self):  # старт игры
        self.greet()
        self.loop()
        
g = Game()
g.start()

-------------------
  Приветсвуем вас  
      в игре       
    морской бой    
-------------------
 формат ввода: x y 
 x - номер строки  
 y - номер столбца 
--------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | O | $ | O | $ | $ | $ |
2 | O | O | O | O | O | O |
3 | O | $ | O | O | O | O |
4 | O | $ | O | O | $ | $ |
5 | O | O | O | O | O | O |
6 | $ | O | $ | O | O | $ |
--------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | O | O | O | O | O | O |
2 | O | O | O | O | O | O |
3 | O | O | O | O | O | O |
4 | O | O | O | O | O | O |
5 | O | O | O | O | O | O |
6 | O | O | O | O | O | O |
--------------------
Ходит пользователь!


Ваш ход:  1 1


Мимо!
--------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | O | $ | O | $ | $ | $ |
2 | O | O | O | O | O | O |
3 | O | $ | O | O | O | O |
4 | O | $ | O | O | $ | $ |
5 | O | O | O | O | O | O |
6 | $ | O | $ | O | O | $ |
--------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | O | O | O | O | O |
2 | O | O | O | O | O | O |
3 | O | O | O | O | O | O |
4 | O | O | O | O | O | O |
5 | O | O | O | O | O | O |
6 | O | O | O | O | O | O |
--------------------
Ходит компьютер!
Ход компьютера: 5 5
Мимо!
--------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | O | $ | O | $ | $ | $ |
2 | O | O | O | O | O | O |
3 | O | $ | O | O | O | O |
4 | O | $ | O | O | $ | $ |
5 | O | O | O | O | . | O |
6 | $ | O | $ | O | O | $ |
--------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | O | O | O | O | O |
2 | O | O | O | O | O | O |
3 | O | O | O | O | O | O |
4 | O | O | O | O | O | O |
5 | O | O | O | O | O | O |
6 | O | O | O 

Ваш ход:  3 3


Мимо!
--------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | O | $ | O | $ | $ | $ |
2 | O | O | O | O | O | O |
3 | O | $ | O | O | O | O |
4 | O | $ | O | O | $ | $ |
5 | O | O | O | O | . | O |
6 | $ | O | $ | O | O | $ |
--------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | O | O | O | O | O |
2 | O | O | O | O | O | O |
3 | O | O | . | O | O | O |
4 | O | O | O | O | O | O |
5 | O | O | O | O | O | O |
6 | O | O | O | O | O | O |
--------------------
Ходит компьютер!
Ход компьютера: 3 1
Мимо!
--------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | O | $ | O | $ | $ | $ |
2 | O | O | O | O | O | O |
3 | . | $ | O | O | O | O |
4 | O | $ | O | O | $ | $ |
5 | O | O | O | O | . | O |
6 | $ | O | $ | O | O | $ |
--------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | O | O | O | O | O |
2 | O | O | O | O | O | O |
3 | O | O | . | O | O | O |
4 | O | O | O | O | O | O |
5 | O | O | O | O | O | O |
6 | O | O | O 

...

Корабль ранен!
--------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | $ | O | $ | X | $ |
2 | . | . | O | O | . | . |
3 | . | $ | . | O | O | . |
4 | O | $ | . | . | $ | X |
5 | O | . | O | O | . | . |
6 | $ | . | $ | O | . | X |
--------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | . | X | X | . | X |
2 | X | . | . | . | . | . |
3 | . | . | . | X | . | O |
4 | . | O | . | . | . | X |
5 | . | . | . | . | O | X |
6 | . | X | X | . | O | O |
--------------------
Ходит пользователь!


Ваш ход:  3 6


Корабль уничтожен!
--------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | $ | O | $ | X | $ |
2 | . | . | O | O | . | . |
3 | . | $ | . | O | O | . |
4 | O | $ | . | . | $ | X |
5 | O | . | O | O | . | . |
6 | $ | . | $ | O | . | X |
--------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | . | X | X | . | X |
2 | X | . | . | . | . | . |
3 | . | . | . | X | . | X |
4 | . | O | . | . | . | X |
5 | . | . | . | . | . | X |
6 | . | X | X | . | . | . |
--------------------
Ходит компьютер!
Ход компьютера: 5 4
Мимо!
--------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | $ | O | $ | X | $ |
2 | . | . | O | O | . | . |
3 | . | $ | . | O | O | . |
4 | O | $ | . | . | $ | X |
5 | O | . | O | . | . | . |
6 | $ | . | $ | O | . | X |
--------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | . | X | X | . | X |
2 | X | . | . | . | . | . |
3 | . | . | . | X | . | X |
4 | . | O | . | . | . | X |
5 | . | . | . | . | . | X |
6

Ваш ход:  4 2


Корабль уничтожен!
--------------------
Пользователь выиграл!
